# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Relational schema design
Week 5 | Lesson 2.1

## A brief overview of relational schema design (20 mins)

- Schema design objectives
- Common considerations
    - Purpose, data, tables
    - Primary keys
    - Attribute types
    - Table relations
- Normalization
    - 1NF; 2NF; 3NF
    - Occasionally, denormalization

## Learning objectives

- Explain principles of relational schema design
- Recognize violations of normal form
- Fit flat dataset to table schema in postgres or sqlite

## Design objectives

- No redundancy
- Integrity ensured


## Consider

- Your data and their purpose
- Your tables
- Your primary keys
- Your types
- Your table relations
- Normalization
- ... and perhaps denormalization

## Purpose, data, tables

How will you use the database? What data go in it?

As a first pass, split up tables by subject.

### Primary keys

Specify column(s) in each table to be a primary key. Remember, these should be:
    - Unique
    - Non-null
    - Permanent
    
Sometimes a "factless" PK is safest.

### Attribute types

Choose these to preserve integrity and increase efficiency.

## Table relations

One-to-many:

![One-to-many example](./assets/images/one_to_many.svg)
 
E.g. one company to many jobs




Many-to-many:

![Many-to-many example](./assets/images/many_to_many.svg)
    
E.g. Jobs to requirements


One-to-one:

![One-to-one example](./assets/images/one_to_one.svg)

E.g. Jobs to job-specific info

> Check: what are examples of one-to-many, many-to-many, and one-to-one relations in an ecommerce context?

## Normalization

First Normal Form (1NF): Atomic values - no domain has sets as elements. I.e. cells have single values, not repeating values.

Example violation:

![1NF violation example](./assets/images/1nf_violation.svg)



Consequence is that values are of the same and correct type for each attribute.


2NF: 1NF, and non-key columns are **functionally dependent** on the full primary key, and not on any subset of it. If:

$$ \forall t,u \in R:$$
$$t[A_1, ..., A_n] = u[A_1, ..., A_n] \implies t[B_1, ..., B_n] = u[B_1, ..., B_m]$$

Then A determines B, that is B is functionally dependent on A:

$$A_1, ..., A_n \rightarrow B_1, ..., B_m$$

Example violation:

![2NF violation example](./assets/images/2nf_violation.svg)

With this violation, you could change a portion of the primary key (e.g. '1' -> '3') and now have different keys referencing different rows: a problem with data integrity.

3NF: 2NF, and non-key columns are functionally dependent **only** on the primary key.

![3NF violation example](./assets/images/3nf_violation.svg)

This again risks inconsistency if some records change but not others.

> Check: can you normalize your ecommerce database example?

## Denormalization

After all that work, one may want to violate the normal forms in order to optimize queries. (For example, have to JOIN many tables may slow performance.) This is a rich topic 
outside our current scope.

## Exercise:
- Postgres or sqlite database creation
- Data exploration 
- Design schema with >= 2 tables
- Create table, import [CSV](./assets/datasets/Eviction_Notices.csv)
- Create new table(s), copy relevant columns

## Some starter code...

`$psql lab33`

``DROP TABLE IF EXISTS evictions_simple;
CREATE TABLE evictions_simple
(eviction_id varchar,
 address varchar,
 city varchar,
 state varchar,
 zip varchar,
 file_date varchar,
 non_payment varchar,
 breach varchar,
 nuisance varchar,
 illegal_use varchar,
 failure_to_sign_renewal varchar,
 access_denial varchar,
 unapproved_subtenant varchar,
 owner_move_in varchar,
 demolition varchar,
 capital_improvement varchar,
 substantial_rehab varchar,
 ellis_act_withdrawal varchar,
 condo_conversion varchar,
 roommate_same_unit varchar,
 other_cause varchar,
 late_payments varchar,
 lead_remediation varchar,
 development varchar,
 good_samaritan_ends varchar,
 constraints varchar,
 constraints_date varchar,
 supervisor_district varchar,
 neighborhood varchar,
 client_location varchar);
``

``COPY evictions_simple FROM '../../assets/datasets/Eviction_Notices.csv' DELIMITER ',' CSV HEADER;``

### Possible solution code:

> Start by recasting the attributes into more appropriate types. Then, as a toy example, let's make a separate table with the neighborhood | supervisor_district values.


`COPY evictions_simple FROM '/Users/your_account/DSI-NYC-1/curriculum/week-05/2.1-schema-design-postgres/assets/datasets/Eviction_Notices.csv' DELIMITER ',' CSV HEADER;

SELECT DISTINCT neighborhood, supervisor_district INTO sd_geo FROM evictions_simple;

ALTER TABLE sd_geo ADD COLUMN geo_id SERIAL PRIMARY KEY;

ALTER TABLE evictions_simple ADD COLUMN geo_id INTEGER;

UPDATE evictions_simple SET geo_id = sd_geo.geo_id FROM sd_geo WHERE evictions_simple.neighborhood = sd_geo.neighborhood AND evictions_simple.supervisor_district = sd_geo.supervisor_district;

ALTER TABLE evictions_simple DROP supervisor_district;

ALTER TABLE evictions_simple DROP neighborhood;`